In [1]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import string
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import WordNetLemmatizer
import malaya
from googletrans import Translator, constants
from nltk.corpus import stopwords

2023-04-25 18:39:27.533020: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 18:39:27.575171: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 18:39:27.575910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 18:39:28.385262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
check compatible Tensorflow version with Tensor

In [2]:
df = pd.read_csv("reviewData.csv")

In [3]:
df.shape

(3499, 11)

In [4]:
df.head()

,Unnamed: 0,username,rating,brand,model,variation,description,delivery_service,product_quality,seller_service,comment
0,0,"""n*****s""","""5""","""Apple""","""Apple iPhone 14 Pro""","""Space Black,128GB""","""iPhone\u00a014\u00a0Pro. Capture incredible d...","""5.0""","""5""","""5.0""","""Performance:High Quality & fast delivery\nQua..."
1,1,"""mumys_baby""","""5""","""Apple""","""Apple iPhone 14 Pro""","""Deep Purple,256GB""","""iPhone\u00a014\u00a0Pro. Capture incredible d...","""5.0""","""5""","""5.0""","""Performance:High\nQuality:Excellent\n\nGood p..."
2,2,"""a*****m""","""5""","""Apple""","""[Pre-order] Apple iPhone 14 Pro [ETA: TBC]""","""Deep Purple,256GB""","""iPhone\u00a014\u00a0Pro. Capture incredible d...","""5.0""","""5""","""5.0""","""Performance:high quality and faster delivery\..."
3,3,"""weeboonhee""","""5""","""Apple""","""Apple iPhone 14 Pro""","""Silver,128GB""","""iPhone\u00a014\u00a0Pro. Capture incredible d...","""5.0""","""5""","""5.0""","""Performance:Perfect\nQuality:As promised\n\nF..."
4,4,"""a*****n""","""5""","""Apple""","""[Pre-order] Apple iPhone 14 Pro [ETA: 6 weeks...","""Deep Purple,128GB""","""iPhone\u00a014\u00a0Pro. Capture incredible d...","""5.0""","""5""","""5.0""","""Order : 22nd Oct\nShipped : 29th Oct\nDeliver..."


In [5]:
df.brand.unique()

array(['"Apple"', '"Vivo"', '"Oppo"', '"Samsung"'], dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3499 entries, 0 to 3498
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3499 non-null   int64 
 1   username          3499 non-null   object
 2   rating            3499 non-null   object
 3   brand             3499 non-null   object
 4   model             3499 non-null   object
 5   variation         3499 non-null   object
 6   description       3499 non-null   object
 7   delivery_service  3499 non-null   object
 8   product_quality   3499 non-null   object
 9   seller_service    3499 non-null   object
 10  comment           3499 non-null   object
dtypes: int64(1), object(10)
memory usage: 300.8+ KB


In [7]:
df.isna().sum()

Unnamed: 0          0
username            0
rating              0
brand               0
model               0
variation           0
description         0
delivery_service    0
product_quality     0
seller_service      0
comment             0
dtype: int64

In [8]:
df.dropna(subset=["comment"], inplace = True)

In [9]:
df.isna().sum()

Unnamed: 0          0
username            0
rating              0
brand               0
model               0
variation           0
description         0
delivery_service    0
product_quality     0
seller_service      0
comment             0
dtype: int64

# Text pre-processing

In [10]:
# removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# remove digit
def remove_digit(text):
    text = re.sub(r'[0-9]+', '', str(text))
    return text

# lower case
def lower_text(text):
    text = text.lower()
    return text

# remove material & comfortability
def remove_mat_comfort(text):
    text = re.sub(r'material:', '', text)
    text = re.sub(r'comfortability:', '', text)
    return text

# removing emoji
def emoji(text):
    emoji = re.compile("[" u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji.sub(r'', text)
    return text
        
# removing line breaks
def line_breaks(text):
    text = re.sub(r'\\n', ' ', text)
    return text
    
# removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern,' ',text)
    return text

# remove quote
def remove_quote(text):
    text = re.sub(r'\"', '', str(text))
    return text


In [11]:
df["username"] = df["username"].apply(remove_quote)
df["rating"] = df["rating"].apply(remove_quote)
df["brand"] = df["brand"].apply(remove_quote)
df["model"] = df["model"].apply(remove_quote)
df["variation"] = df["variation"].apply(remove_quote)
df["description"] = df["description"].apply(remove_quote)
df["delivery_service"] = df["delivery_service"].apply(remove_quote)
df["product_quality"] = df["product_quality"].apply(remove_quote)
df["seller_service"] = df["seller_service"].apply(remove_quote)

In [12]:
df["comment"] = df["comment"].apply(strip_html)
df["comment"] = df["comment"].apply(remove_digit)
df["comment"] = df["comment"].apply(lower_text)
df["comment"] = df["comment"].apply(remove_mat_comfort)
df["comment"] = df["comment"].apply(emoji)
df["comment"] = df["comment"].apply(line_breaks)
df["comment"] = df["comment"].apply(remove_between_square_brackets)
df["comment"] = df["comment"].apply(denoise_text)
df["comment"] = df["comment"].apply(remove_special_characters)


In [13]:
df.head()

,Unnamed: 0,username,rating,brand,model,variation,description,delivery_service,product_quality,seller_service,comment
0,0,n*****s,5,Apple,Apple iPhone 14 Pro,"Space Black,128GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance high quality fast delivery qual...
1,1,mumys_baby,5,Apple,Apple iPhone 14 Pro,"Deep Purple,256GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance high quality excellent good prod...
2,2,a*****m,5,Apple,[Pre-order] Apple iPhone 14 Pro [ETA: TBC],"Deep Purple,256GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance high quality and faster delivery ...
3,3,weeboonhee,5,Apple,Apple iPhone 14 Pro,"Silver,128GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance perfect quality as promised fast...
4,4,a*****n,5,Apple,[Pre-order] Apple iPhone 14 Pro [ETA: 6 weeks ...,"Deep Purple,128GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,order nd oct shipped th oct delivered t...


In [14]:
df["comment"].head(20)

0      performance high quality   fast delivery qual...
1      performance high quality excellent  good prod...
2      performance high quality and faster delivery ...
3      performance perfect quality as promised  fast...
4      order   nd oct shipped   th oct delivered   t...
5      fast delivery  save extra for the iphone  ver...
6      performance ori product from apple \udc\udfe ...
7      performance terbaikk  terbaik  penghantaran p...
8      performance great quality great  received ite...
9      performance harap ok quality harap ok  hi sel...
10     quality best performance best  fast delivery ...
11     alhamdulillah selamat sampai dan berfungsi de...
12     performance good quality good  fast delivery ...
13     terima kasih seller    saya beli    alhamduli...
14     quality smooth to touch  pre ordered on    sh...
15     fast delivery   alot bubble wrap in the box  ...
16     performance very good quality very good  beau...
17     performance as expected  really smooth  w

# Tokenization

In [15]:
def tokens_text(sentences, is_lower_case=False):
    tokenizer = malaya.tokenizer.Tokenizer()
    tokens = tokenizer.tokenize(sentences)
    new_tokens = [token.strip() for token in tokens]
    return new_tokens

# detokenization
def toString(tokens):
    global string #
    for token in tokens:
        string = " ".join(tokens)
    return string

In [16]:
tokens = df["comment"].apply(tokens_text)
print(tokens)

0       [performance, high, quality, fast, delivery, q...
1       [performance, high, quality, excellent, good, ...
2       [performance, high, quality, and, faster, deli...
3       [performance, perfect, quality, as, promised, ...
4       [order, nd, oct, shipped, th, oct, delivered, ...
                              ...                        
3494            [super, fast, delivery, thank, u, seller]
3495                                                   []
3496                                                   []
3497                                                   []
3498                                                   []
Name: comment, Length: 3499, dtype: object


# Rojak Lemmatization

In [17]:
# create rojak dictionary
rojak_dict = {
    "sye": "saya",
    "sya": "saya",
    "yg": "yang",
    "kau": "engkau",
    "korang": "kalian",
    "kitorang": "kami",
    "diorang": "mereka",
    "cun": "cantik",
    "chun": "cantik",
    "best": "terbaik",
    
    "tapau": "bungkus",
    "tapao": "bungkus",
    "lu": "kamu",
    "cincai": "apa saja",
    "tu": "situ",
    "bojio": "tidak jemput",
    "fuyoh": "wah",
    "dei": "kawan",
    "macha": "kawan",
    "on": "jom",
    
    "pokai": "miskin",
    "oky": "memuaskan",
    "tp": "tetapi",
    "x": "tidak",
    "diorder": "dibeli",
    "ape2": "apa-apa",
    "kat": "di",
    "cam": "macam",
    "je": "sahaja",
    "i": "saya",
    
    "order": "beli",
    "mne": "mana",
    "tau": "tahu",
    "mmg": "memang",
    "bleh": "boleh",
    "uishh": "wah", 
    "sgtt": "sangat",
    "lg": "lagi",
    "sbb": "sebab",
    "dr": "dari",
    
    "nak": "ingin",
    "nk": "hendak", 
    "da": "sudah", 
    "brg": "barang", 
    "yg": "yang",
    "xsmpai": "tidak sampai", 
    "trus": "terus",
    "smp": "sampai",
    "dgn": "dengan",
    "tpi": "tetapi",
    
    "xda": "tidak ada",
    "arini": "hari ini",
    "klau": "kalau",
    "bgus": "bagus",
    "utk": "untuk",
    "pun": "juga",
    "byk": "banyak",
    "sebok": "sibuk",
    "tkda": "tidak ada",
    "xtry": "belum cuba",
    
    "sgt": "sangat",
    "bleh": "boleh",
    "ckp": "cukup",
    "ade": "ada",
    "pon": "pun",
    "blh": "boleh",
    "kli": "kali",
    "xingat": "tidak ingat",
    "smpi": "sampai",
    "pn": "pun",
    
    "dlm": "dalam",
    "dpt": "dapat",
    "wane": "warna",
    "skit": "sedikit",
    "bli": "beli",
    "bkn": "bukan",
    "sbg": "sebagai",
    "dh": "sudah",
    "maw": "hendak",
    "rs": "rasa",
    
    "tak": "tidak",
    "jge": "menjag a",
    "tk": "tidak",
    "sy": "saya",
    "ptg": "petang",
    "bisnes": "perniagaan",
    "tngok": "tengok",
    "sngt": "sangat",
    "lps": "lepas",
    "kbtulan": "kebetulan",
    
    "lgsg": "langsung",
    "jd": "jadi",
    "xdpt": "tidak dapat",
    "ngam": "sesuai",
    "ska": "suka",
    "jew": "sahaja ini",
    "blm": "belum",
    "hensem": "kacak",
    "ksut": "kasut",
    "cpt": "cepat",
    
    #100 up
    
    "ltk": "letak",
    "pki": "memakai",
    "bole": "boleh",
    "hri": "hari",
    "jgn": "jangan",
    "ank": "anak",
    "hri": "hari",
    "lipstik": "gincu",
    "betol": "betul",
    "slalu": "selalu",
    
    "nnti": "nanti",
    "bru": "baru",
    "bsh": "basah",
    "gmbr": "gambar",
    "chntek": "cantik",
    "bwa": "membawa",
    "pnts": "pantas",
    "cantek": "cantik",
    "npk": "nampak",
    "bg": "bagi",
    
    "kwn": "kawan",
    "terbaek": "terbaik",
    "nnt": "nanti",
    "sdh": "sudah",
    "lembt": "lembut",
    "tk pe": "tidak mengapa",
    "hb": "hari besok",
    "dkt": "dekat",
    "bsuh": "basuh",
    "ngam": "sesuai",
    
    "sblum": "sebelum ini",
    "brpa": "berapa",
    "smlm": "semalam",
    "xbole": "tidak boleh",
    "kaler": "warna",
    "akn": "akan",
    "drpd": "daripada",
    "hntr": "hantar",
    "cri": "cari",
    "vdio": "video",
    
    "lbt": "lembut",
    "sshnya": "susahnya",
    "pnjg": "panjang",
    "didlm": "di dalam",
    "kreta": "kereta",
    "hrp": "harap",
    "sdkit": "sedikit",
    "nps": "nipis",
    "bintg": "bintang",
    "prnh": "pernah",
    
    "blastik": "plastik",
    "qlti": "qualiti",
    "smua": "semua",
    "tdie": "tadi",
    "betol": "betulnya", 
    "bsr": "besar",
    "klau": "kalau",
    "akn": "akan",
    "xpshati": "tidak puas hati",
    "sbnrnya": "sebenarnya",
    
    "dtrm": "diterima",
    "cpt": "cepat",
    "pnts": "pantas",
    "tebl": "tebal",
    "hrga": "harga",
    "tidor": "tidur",
    "jgn": "jangan",
    "skrg": "sekarang",
    "xpasti": "tidak pasti",
    "sedp": "sedap",
    
    "jap": "sos",
    "kwn": "kawan-kawan",
    "rsa": "rasa",
    "btol": "botol",
    "dah": "sudah",
    "slmat": "selamat",
    "recomen": "cadangan",
    "ptama": "pertama",
    "supok": "support",
    "mkn": "mungkin",
     
    "pkai": "pakai",
    "ape": "apa",
    "dpt": "dapat",
    "mahel": "mahal",
    "saller": "penjual",
    "hrp": "harap",
    "bli": "beli",
    "msti": "mesti",
    "mdh": "mudah",
    "ptng": "penting",
    
    "prduk": "produk",
    "wgi": "wangi",
    "cpon": "kupon",
    "bgus": "bagus",
    "kmas": "kemas",
    "agk": "agak",
    "bhgian": "bahagian",
    "bwah": "bawah",
    "jhitan": "jahitan",
    "bwh": "bawah",
    
    #200 up
    
    "nye": "nya",
    "vd": "rakaman",
    "lmbt": "lembut",
    "dlm": 'dalam',
    "tdk": "tidak",
    "lpstu": "lepas itu",
    "wlaupun": "walaupun",
    "kasi": "bagi",
    "xngam": "tidak sesuai",
    "kaler": "warna",
    
    "bsar": "besar",
    "kecik": "kecil",
    "brt": "berat",
    "syg": "sayang",
    "brisi": "berisi",
    "bju": "baju",
    "rsa": "rasa",
    "lngar": "longgar",
    "pns": "panas",
    "msih": "masih",
    
    "sk": "suka",
    "priz": "harga",
    "qlti": "qualiti",
    "byk": "banyak",
    "xssuai": "tidak sesuai",
    "ambik": "ambil",
    "nmpk": "nampak",
    "tidoq": "tidur",
    "lubng": "lubang",
    "butng": "butang",
    
    "bnyk": "banyak",
    "mf": "maaf",
    "ringn": "ringan",
    "mrh": "murah",
    "tggi": "tinggi",
    "da": "sudah",
    "tgi": "tinggi", 
    "saller": "penjual",
    "bgs": "bagus",
    "bl": "boleh",
    
    "mlm": "malam",
    "fhm": "faham",
    "skli": "sekali",
    "degn": "dengan",
    "sjk": "sejuk",
    "org": "orang",
    "lr": "luar",
    "sp": "sampai",
    "dg": "dengan",
    "ringn": "ringan",
    
    "cpt": "cepat",
    "kcik": "kecil",
    "klau": "kalau",
    "ambik": "ambil",
    "bju": "baju",
    "je": "sangat",
    "dri": "dari",
    "membzir": "membazir",
    "xbest": "tidak bagus",
    "ktor": "kotor",
    
    "yg": "yang",
    "pch": "pecah",
    "slm": "selamat",
    "beso": "besok",
    "bgus": "bagus",
    "nk": "hendak",
    "sndiri": "sendiri",
    "kpd": "kepada",
    "ad": "ada",
    "ktk": "kotak",
    
    "bile": "bila",
    "bnyk": "banyak",
    "tgk": "tengok",
    "refun": "bayaran balik",
    "hntr": "hantar",
    "ptg": "petang",
    "walaupon": "walaupun",
    "boik": "baik",
    "lgi": "lagi",
    "lebeh": "lebih",
    
    "tmbh": "tambah",
    "funcsion": "fungsi",
    "rasenye": "rasanya",
    "lm": "lama",
    "smua": "semua",
    "xrgi": "tidak rugi",
    "xtau": "tidak tahu",
    "mnman": "minuman",
    "ckp": "cakap",
    "gune": "guna",
    
    "bukak": "buka",
    "sluru": "seluruh",
    "maklmt": "maklumat",
    "keadan": "keadaan",
    "wreping": "pembungkusan",
    "trm": "terima",
    "byk": "banyak",
    "trm": "terima",
    "xrsak": "tidak rosak",
    "kt": "dekat",
    
    #300 up
    
    "shj": "sahaja",
    "keja": "kerja",
    "pace": "parcel",
    "jg": "jangan", 
    "kemus": "kemas",
    "penhantrn": "penghantaran",
    "leju": "laju",
    "pon": "juga",
    "blum": "belum",
    "lmpu": "lampu",
    
    "msa": "masa",
    "kcctan": "kecacatan",
    "blom": "belum",
    "slmet": "selamat",
    "beik": "baik",
    "berfgsi": "berfungsi",
    "lanjt": "lanjut",
    "pnts": "pantas",
    "kna": "kena",
    "ofer": "diskaun",
    
    "msih": "masih",
    "mrh": "merah",
    "dngan": "dengan",
    "ltk": "letak",
    "ngr": "negara",
    "ja": "sahaja",
    "thnlama": "tahan lama",
    "ringn": "ringan",
    "sdp": "sedap",
    "ssuai": "sesuai",
    
    "kta": "kita",
    "cpt": "cepat",
    "dkt": "dekat",
    "klit": "kulit",
    "slmt": "selamat",
    "berklti": "berkualiti",
    "psmen": "posmen",
    "tngn": "tangan",
    "diset": "ditetapkan",
    "xnmpak": "tidak nampak",
    
    "dr": "dari",
    "ode": "beli",
    "rasenye": "rasanya",
    "boss": "bos",
    "ksh": "kasih",
    "cincai": "apa-apa sahajalah",
    "fuyoh": "wah",
    "masyuk": "kaya",
    "tapao": "bawa pulang",
    "walao": "wah",
    
    "maa": "pun",
    "awek": "perempuan",
    "cun": "cantik",
    "meh": "kah",
    "diam": "sila senyap",
    "fong fei kei": "membatalkan janji temu",
    "jom": "marilah",
    "kautim": "selesai",
    
    "wahlao": "wah",
    "lembat": "lambat",
    "licn": "licin",
    "bsr": "besar",
    "sof": "lembut",
    "klt": "kualiti",
    "stuk": "stok", 
    "tebul": "tebal",
    
    "designnya": "reka bentuknya",
    "bnyk": "banyak",
    "xlebar": "tidak lebar",
    "prmsi": "promosi",
    "ditrma": "diterima",
    "condision": "keadaan",
    "ps hati": "puas hati",
    "selese": "selesa",
    "suke": "suka",
    "eluk": "elok",
    
    "rndh": "rendah",
    "pakaging": "pembungkusan",
    "kondision": "keadaan",
    "poduk": "produk",
    "agk": "agak",
    "hrgasal": "harga asal",
    "dwsa": "dewasa",
    "lmbat": "lambat", 
    "mahel": "mahal",
    "spruh": "separuh",
    
    "ingt": "ingat",
    "sangt": "sangat",
    "mmta mf": "minta maaf", 
    "brkaitn": "berkaitan",
    "menghrp": "mengharap",
    "rmh": "rumah",
    "gune": "guna",
    "pantes": "pantas",
    "xkring": "tidak kering",
    "smua": "semua",
    
    #400 up
    
    "srp": "serap",
    "sblm": "sebelum",
    "slps": "selepas",
    "hmpir": "hampir",
    "msti": "mesti",
    "sbuln": "sebulan",
    "xnak": "tidak mahu",
    "mslh": "masalah",
    "mmhmi": "memahami",
    "wlpn": "walaupun",
    
    "mtrial": "bahan",
    "xkaitan": "tidak berkaitan",
    "ckp": "cukup",
    "kurg": "kurang",
    "sbg": "sebagai",
    "bolh": "boleh",
    "xgood": "tidak bagus",
    "cntik": "cantik", 
    "dpt": "dapat",
    "handfone": "telepon bimbit", 
    
    "lj": "laju",
    "ltk": "letak",
    "mknn": "makanan",
    "byk": "banyak",
    "xbrpe": "tidak berapa", 
    "tersyg": "tersayang",
    "suke": "suka",
    "pm": "bersembang secara peribadi",
    "bwk": "bawa ke",
    "jhtnye": "jahitnya",
    
    "besr": "besar", 
    "hr": "hari",
    "sngt": "sangat", 
    "saller": "penjual", 
    "gam": "gum",
    "buke": "buka", 
    "slmt": "selamat",
    "jg": "juga", 
    "suda": "sudah",
    "smua": "semua", 
    
    "nmpk": "nampak",
    "msih": "masih", 
    "pkai": "pakai", 
    "slsa": "selesa",
    "dihntr": "dihantar",
    "mdh": "mudah", 
    "sangt": "sangat", 
    "untk": "untuk", 
    "sekli": "sekali", 
    "nips": "nipis",
    
    "lembt": "lembut", 
    "sjk": "sejuk", 
    "skrng": "sekarang", 
    "xsame": "tidak sama", 
    "knp": "kenapa", 
    "tpi": "tetapi", 
    "hrga": "harga", 
    "bln": "bulan", 
    "trimksih": "terima kasih", 
    "ckp": "cukup", 
    
    "koton": "kapas", 
    "hdh": "hadiah", 
    "hrian": "harian", 
    "awk": "awak", 
    "krng": "kurang", 
    "brpkali": "berapa kali", 
    "rpeat": "ulang", 
    "ank": "anak", 
    "akn": "akan", 
    "rekemen": "cadangan", 
    
    "tk": "tidak", 
    "bsr": "besar", 
    "hntar": "hantar", 
    "trng": "terang", 
    "dpn": "depan", 
    "blkng": "belakang", 
    "lbngnye": "lubangnya", 
    "hejau": "hijau",
    "lbih": "lebih",
    "ambiklah": "ambil lah",
    
    "hbis": "habis", 
    "xpns": "tidak panas", 
    "hlai": "helai", 
    "abg": "abang", 
    "umo": "umur", 
    "hrslah": "haruslah", 
    "vedio": "rakaman", 
    "krja": "kerja", 
    "bpatutn": "berpatutan", 
    "trimaksih": "terima kasih", 
    
    "staf": "kakitangan", 
    "vakcine": "vaksin", 
    "ckpnye": "cukupnya", 
    "cuace": "cuaca", 
    "commel": "comel", 
    "blilg": "beli lagi", 
    "jns": "jenis",
    "siez": "saiz", 
    "tq": "terima kasih", 
    "tdk": "tidak",
    
    "mcm": "macam",
    "mau": "mahu",
    "pangsai": "buang air besar",
    "ni": "ini",
    "la": "",
    "lah": "",
    "u": "kamu",
    "wor": "",
    
    "okey": "baik",
    "ok": "baik",
    "okay": "baik",
    "nd": "and",
    "oct": "october",
    "th": "the"
}

In [18]:
def rojak_removal(tokens):
    for token in tokens:
        if token in rojak_dict:
            new_token = rojak_dict[token]
            for i in range(len(tokens)):
                if tokens[i] == token:
                    tokens[i] = new_token
    return tokens

In [19]:
no_rojak = tokens.apply(rojak_removal)
print(no_rojak)

0       [performance, high, quality, fast, delivery, q...
1       [performance, high, quality, excellent, good, ...
2       [performance, high, quality, and, faster, deli...
3       [performance, perfect, quality, as, promised, ...
4       [beli, and, october, shipped, the, october, de...
                              ...                        
3494         [super, fast, delivery, thank, kamu, seller]
3495                                                   []
3496                                                   []
3497                                                   []
3498                                                   []
Name: comment, Length: 3499, dtype: object


# Translation

In [20]:
def translate_text(tokens):
    translator = Translator()
    new_tokens = []
    for token in tokens: 
        translation = translator.translate(token, dest="en")
        new_tokens.append(translation.text)
    return new_tokens

In [21]:
translated = no_rojak.apply(translate_text)
translated = translated.apply(toString)
print(translated)

0       performance high quality fast delivery quality...
1       performance high quality excellent good produc...
2       performance high quality and faster delivery t...
3       performance perfect quality as promised fast d...
4       buy and october shipped the october delivered ...
                              ...                        
3494                 super fast delivery thank You seller
3495                 super fast delivery thank You seller
3496                 super fast delivery thank You seller
3497                 super fast delivery thank You seller
3498                 super fast delivery thank You seller
Name: comment, Length: 3499, dtype: object


In [22]:
# convert to lower case
translated_lwr = translated.apply(lower_text)
print(translated_lwr)

0       performance high quality fast delivery quality...
1       performance high quality excellent good produc...
2       performance high quality and faster delivery t...
3       performance perfect quality as promised fast d...
4       buy and october shipped the october delivered ...
                              ...                        
3494                 super fast delivery thank you seller
3495                 super fast delivery thank you seller
3496                 super fast delivery thank you seller
3497                 super fast delivery thank you seller
3498                 super fast delivery thank you seller
Name: comment, Length: 3499, dtype: object


# Remove Stopwords & lemmatization

In [23]:
tokenizer2 = ToktokTokenizer()
nltk.download('stopwords')
stopword_lis2 = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/user2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
stop = set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer2.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_lis2]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_lis2]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    lemmaArray = []
    for word in text.split():
        lemmaWord = ' '.join([lemmatizer.lemmatize(word)])
        lemmaArray.append(lemmaWord)
    return lemmaArray



{'an', "you've", 'itself', 'its', 'mustn', 'he', 'not', "you'll", 'on', 'ours', 'from', 'her', 'and', "hadn't", 'was', 'but', 'few', 'him', "didn't", 'as', 'own', 're', 'mightn', 'do', "mustn't", 'does', 'their', 'no', "that'll", 'each', 'to', 'after', 'of', "couldn't", 'where', 'your', 'whom', 'them', 'between', 'there', 'yourselves', 'in', 'd', "weren't", 'ain', "wouldn't", 'some', 'all', 'the', 'that', "hasn't", 'himself', 'out', 'at', 'i', 'hers', 'being', 'against', 'ma', 'under', 'those', 'above', 'shouldn', 'wasn', 'they', "shouldn't", 'me', 'are', 'doing', 'when', 'hasn', 'needn', 'a', 'most', "she's", 'again', 'then', 'very', 'can', "won't", 'am', 'while', 'over', 'for', 'she', 'hadn', 'has', 'or', 'is', 'isn', 'why', 'if', 'will', 've', 'aren', 'below', 'herself', 'had', 'we', 'my', 'o', 'because', "you'd", 'theirs', "shan't", 'this', 'myself', 'here', 'yours', "doesn't", 'down', 'did', 'weren', 'into', 'ourselves', 'our', 'm', "isn't", "mightn't", 'such', 'with', 'who', 'be'

In [25]:
translated = translated_lwr
translated = translated.apply(remove_stopwords)
print(translated)

0       performance high quality fast delivery quality...
1       performance high quality excellent good produc...
2       performance high quality faster delivery time ...
3       performance perfect quality promised fast deli...
4       buy october shipped october delivered october ...
                              ...                        
3494                     super fast delivery thank seller
3495                     super fast delivery thank seller
3496                     super fast delivery thank seller
3497                     super fast delivery thank seller
3498                     super fast delivery thank seller
Name: comment, Length: 3499, dtype: object


In [26]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/user2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
lemmaWord = translated.apply(lemmatization)
lemmaWord = lemmaWord.apply(toString)
lemmaWord

0       performance high quality fast delivery quality...
1       performance high quality excellent good produc...
2       performance high quality faster delivery time ...
3       performance perfect quality promised fast deli...
4       buy october shipped october delivered october ...
                              ...                        
3494                     super fast delivery thank seller
3495                     super fast delivery thank seller
3496                     super fast delivery thank seller
3497                     super fast delivery thank seller
3498                     super fast delivery thank seller
Name: comment, Length: 3499, dtype: object

In [28]:
df["comment"] = lemmaWord

In [29]:
# still find out that remain alot of slash "/"
# removing slash
def slash_remove(text):
    text = re.sub(r'\\', '', text)
    return text
    

In [30]:
df["comment"] = df["comment"].apply(strip_html)
df["comment"] = df["comment"].apply(remove_digit)
df["comment"] = df["comment"].apply(lower_text)
df["comment"] = df["comment"].apply(remove_mat_comfort)
df["comment"] = df["comment"].apply(emoji)
df["comment"] = df["comment"].apply(line_breaks)
df["comment"] = df["comment"].apply(remove_between_square_brackets)
df["comment"] = df["comment"].apply(denoise_text)
df["comment"] = df["comment"].apply(remove_special_characters)
df["comment"] = df["comment"].apply(slash_remove)

In [31]:
df.head()

,Unnamed: 0,username,rating,brand,model,variation,description,delivery_service,product_quality,seller_service,comment
0,0,n*****s,5,Apple,Apple iPhone 14 Pro,"Space Black,128GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance high quality fast delivery quality...
1,1,mumys_baby,5,Apple,Apple iPhone 14 Pro,"Deep Purple,256GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance high quality excellent good produc...
2,2,a*****m,5,Apple,[Pre-order] Apple iPhone 14 Pro [ETA: TBC],"Deep Purple,256GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance high quality faster delivery time ...
3,3,weeboonhee,5,Apple,Apple iPhone 14 Pro,"Silver,128GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,performance perfect quality promised fast deli...
4,4,a*****n,5,Apple,[Pre-order] Apple iPhone 14 Pro [ETA: 6 weeks ...,"Deep Purple,128GB",iPhone\u00a014\u00a0Pro. Capture incredible de...,5.0,5,5.0,buy october shipped october delivered october ...


In [32]:
df.to_csv(r'processedData.csv', index=False)
print('Check point')

Check point
